In [28]:
import gurobipy as gp
from itertools import product
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from utils  import division

## 1 Modèle de base des centrales thermiques

### 1.1 Planification journalière du parc thermique

Variables de décision :
Pour une type de centrale $X \in [A,B,C]$, et une heure de la journée $t\in {1,\dots,24}$, on définit :
$$
N_t^{(X)} = \text{Nombre de centrales } X \text{allumées à } t\text{ h, (ENTIER)}
$$
$$
P_t^{(X)} = \text{Puissance totale produite par les centrales } X \text{ à } t\text{ h, (CONTINUE)}
$$
Contraintes :
$$
N_t^{(X)} P_{min}^{(X)} \leq P_t^{(X)} \leq N_t^{(X)} P_{max}^{(X)} \text{ ,  Contraintes sur la puissance totale de chaque centrale}
$$
$$
0 \leq N_t^{(X)} \leq N^{(X)} \text{   ,   Contraintes sur le nombre de centrales allumées possible}
$$
$$
\forall t, \sum_X P_t^{(X)} = D_t \text{  ,  Contrainte équilibre offre-demande}
$$
Objectif :
$$
\text{Minimiser} \sum_X \sum_T P_t^{(X)} C_{MWh}^{(X)}
$$

In [29]:
from utils import Centrale

dict_Thermique = {
    "A" : Centrale(
    name="A",
    N=12,
    Pmax=2000,
    Pmin=850,
    Cmwh=1.5
    ),
    "B" : Centrale(
    name="B",
    N=10,
    Pmax=1750,
    Pmin=1250,
    Cmwh=1.38
    ),
    "C" : Centrale(
    name="C",
    N=5,
    Pmax=4000,
    Pmin=1500,
    Cmwh=2.75
    )
}
consommation = np.array([15,15,15,15,15,15,30,30,30,25,25,25,25,25,25,40,40,40,27,27,27,27,27,27])*1000
heure = np.arange(24)

In [30]:
from utils import variable_decision_thermique, contraintes_equilibre, contraintes_puissance_thermique

model = gp.Model(name="1.1")
dict_N, dict_P = variable_decision_thermique(model=model,dict_Thermique=dict_Thermique)
contraintes_puissance_thermique(model,dict_N,dict_P,dict_Thermique)
contraintes_equilibre(model,dict_P,dict_Thermique,consommation)
model.setObjective(gp.quicksum([dict_P[element[0],element[1]] * dict_Thermique[element[0]].Cmwh for element in product(dict_Thermique,range(24))]), gp.GRB.MINIMIZE)

In [31]:
model.optimize()
print(f"Coût : {model.ObjVal}")

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 168 rows, 144 columns and 360 nonzeros
Model fingerprint: 0xf31da896
Variable types: 72 continuous, 72 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 3e+00]
  Bounds range     [5e+00, 1e+01]
  RHS range        [2e+04, 4e+04]
Found heuristic solution: objective 1235375.0000
Presolve removed 162 rows and 139 columns
Presolve time: 0.02s
Presolved: 6 rows, 5 columns, 14 nonzeros
Found heuristic solution: objective 881275.00000
Variable types: 2 continuous, 3 integer (0 binary)

Root relaxation: objective 8.694000e+05, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 869400.000    0    1 881275.000 869400.000  1.35%     -    

In [32]:
from turtle import color
import plotly.graph_objects as go
from utils_viz import df_results, Results_viz
df = df_results(dict_N,dict_P,dict_Thermique,consommation)
fig = Results_viz(df,dict_Thermique)
A = 1
fig.update_layout(
    title = f"Répartition de la production électrique dans la journée - {A}"
)
fig.show()
fig.write_html(f"HTML/Répartition de la production électrique dans la journée - {A}.html")

In [33]:
model_relax = model.relax()
model_relax.optimize()
print(f"Coût : {model_relax.ObjVal}")

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 168 rows, 144 columns and 360 nonzeros
Model fingerprint: 0x635b27b9
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 3e+00]
  Bounds range     [5e+00, 1e+01]
  RHS range        [2e+04, 4e+04]
Presolve removed 153 rows and 99 columns
Presolve time: 0.01s
Presolved: 15 rows, 45 columns, 45 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.5266000e+05   1.743750e+04   0.000000e+00      0s
      15    8.6940000e+05   0.000000e+00   0.000000e+00      0s

Solved in 15 iterations and 0.01 seconds (0.00 work units)
Optimal objective  8.694000000e+05
Coût : 869400.0


## 2 Coût d'opération

### 2.1 Coût de fonctionnement

Pour ce problème on modifie la fonction objectif.<br>
Objectif :
$$
\text{Minimiser} \sum_X \sum_T (P_t^{(X)} - P_{min}^{(X)}N_t^{(X)}) C_{MWh}^{(X)} + N_t^{(X)}C_{base}^{(X)}
$$

In [34]:
from utils import Centrale2

dict_Thermique = {
    "A" : Centrale2(
    name="A",
    N=12,
    Pmax=2000,
    Pmin=850,
    Cmwh=2,
    Cstart=2000,
    Cbase=1000
    ),
    "B" : Centrale2(
    name="B",
    N=10,
    Pmax=1750,
    Pmin=1250,
    Cmwh=1.3,
    Cstart=1000,
    Cbase=2600
    ),
    "C" : Centrale2(
    name="C",
    N=5,
    Pmax=4000,
    Pmin=1500,
    Cmwh=3,
    Cstart=500,
    Cbase=3000
    )
}

In [35]:
model = gp.Model(name="2.1")
dict_N, dict_P = variable_decision_thermique(model,dict_Thermique)
contraintes_puissance_thermique(model,dict_N=dict_N,dict_P=dict_P,dict_Thermique=dict_Thermique)
contraintes_equilibre(model,dict_P,dict_Thermique,consommation)

model.setObjective(gp.quicksum([dict_N[element[0],element[1]] * (dict_Thermique[element[0]].Cbase - dict_Thermique[element[0]].Pmin*dict_Thermique[element[0]].Cmwh) + dict_P[element[0],element[1]] * dict_Thermique[element[0]].Cmwh for element in product(dict_Thermique,range(24))]), gp.GRB.MINIMIZE)

model.optimize()
print(f"Coût : {model.ObjVal}")

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 168 rows, 144 columns and 360 nonzeros
Model fingerprint: 0x7066cec8
Variable types: 72 continuous, 72 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 2e+03]
  Bounds range     [5e+00, 1e+01]
  RHS range        [2e+04, 4e+04]
Found heuristic solution: objective 1271875.0000
Presolve removed 162 rows and 139 columns
Presolve time: 0.03s
Presolved: 6 rows, 5 columns, 14 nonzeros
Found heuristic solution: objective 987500.00000
Variable types: 2 continuous, 3 integer (0 binary)
Found heuristic solution: objective 985950.00000

Root relaxation: objective 9.787500e+05, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 978750.000

In [36]:
from utils_viz import df_results2
df = df_results2(dict_N=dict_N,dict_P=dict_P,dict_XX=dict_Thermique,consommation=consommation)
fig = Results_viz(df,dict_Thermique)
A = "2.1"
fig.update_layout(
    title = f"Répartition de la production électrique dans la journée - {A}"
)
fig.show()
fig.write_html(f"HTML/Répartition de la production électrique dans la journée - {A}.html")

### 2.2 Coût du démarrage

Pour ce problème on modifie les variables de décisions et on y adapte les contraintes.

Variables de décision :
Pour une type de centrale $X \in [A,B,C]$, et une heure de la journée $t\in {1,\dots,24}$, on définit :
$$
N_t^{(X)} = \text{Nombre de centrales } X \text{allumées à } t\text{ h, (ENTIER)}
$$
$$
N_{start,t}^{(X)} = \text{Nombre de centrales } X \text{démarrées à } t\text{ h, (ENTIER)}
$$
$$
P_t^{(X)} = \text{Puissance totale produite par les centrales } X \text{ à } t\text{ h, (CONTINUE)}
$$
Contraintes (par convention $N_{-1}^{(X)}=0$):
$$
N_t^{(X)} P_{min}^{(X)} \leq P_t^{(X)} \leq N_t^{(X)} P_{max}^{(X)} \text{ ,  Contraintes sur la puissance totale de chaque centrale}
$$
$$
N_{start,t}^{(X)} \leq N_t^{(X)} \leq N_{start,t}^{(X)}+N_{t-1}^{(X)} \text{   ,   Contraintes sur le nombre de centrales allumées possible}
$$
$$
0 \leq N_{start,t}^{(X)} \leq N^{(X)}-N_{t-1}^{(X)} \text{   ,   Contraintes sur le nombre de centrales démarrable possible}
$$
$$
\sum_X P_t^{(X)} = D_t \text{  ,  Contrainte équilibre offre-demande}
$$
Objectif :
$$
\text{Minimiser} \sum_X \sum_T (P_t^{(X)} - P_{min}^{(X)}N_t^{(X)}) C_{MWh}^{(X)} + N_t^{(X)}C_{base}^{(X)} + N_{start,t}^{(X)}C_{start}^{(X)}
$$

In [37]:
from utils import variable_decision_thermique_avec_demarrage, contraintes_demarrage

model = gp.Model(name="2.2")

dict_N, dict_Nstart, dict_P = variable_decision_thermique_avec_demarrage(model,dict_Thermique)

model.setObjective(gp.quicksum([dict_N[element[0],element[1]] * (dict_Thermique[element[0]].Cbase - dict_Thermique[element[0]].Pmin*dict_Thermique[element[0]].Cmwh) + dict_P[element[0],element[1]] * dict_Thermique[element[0]].Cmwh + dict_Nstart[element[0],element[1]]*dict_Thermique[element[0]].Cstart for element in product(dict_Thermique,range(24))]), gp.GRB.MINIMIZE)

contraintes_demarrage(model,dict_N,dict_Nstart,dict_Thermique,cyclique=False)
contraintes_puissance_thermique(model,dict_N,dict_P,dict_Thermique)
contraintes_equilibre(model,dict_P,dict_Thermique,consommation)

model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 312 rows, 216 columns and 714 nonzeros
Model fingerprint: 0xafd71f6e
Variable types: 72 continuous, 144 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 2e+03]
  Bounds range     [5e+00, 1e+01]
  RHS range        [5e+00, 4e+04]
Found heuristic solution: objective 1474375.0000
Presolve removed 108 rows and 30 columns
Presolve time: 0.00s
Presolved: 204 rows, 186 columns, 522 nonzeros
Variable types: 42 continuous, 144 integer (0 binary)
Found heuristic solution: objective 1378930.0000

Root relaxation: objective 1.011257e+06, 52 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1011257.14    0   26 1378930.00 1011257.14  26.7% 

In [38]:
from utils_viz import df_results22
df = df_results22(dict_N=dict_N,dict_P=dict_P,dict_Nstart=dict_Nstart,dict_XX=dict_Thermique,consommation=consommation)
fig = Results_viz(df,dict_Thermique)
A = "2.2"
fig.update_layout(
    title = f"Répartition de la production électrique dans la journée - {A}"
)
fig.show()
fig.write_html(f"HTML/Répartition de la production électrique dans la journée - {A}.html")

## 3 Réserve de puissance

Pour intégrer la réserve de puissance on ajoute la contrainte :
$$
\sum_X N_t^{(X)}P_{max}^{(X)}\geq D_t\times 1,15
$$

In [39]:
from utils import contraintes_reserve_de_puissance

contraintes_reserve_de_puissance(model,dict_N,dict_Thermique,consommation)

model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 336 rows, 216 columns and 786 nonzeros
Model fingerprint: 0x11b5f8ee
Variable types: 72 continuous, 144 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 2e+03]
  Bounds range     [5e+00, 1e+01]
  RHS range        [5e+00, 5e+04]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint Réserve_de_puissance_à_15 by 4500.000000000

Found heuristic solution: objective 1316565.0000
Presolve removed 108 rows and 30 columns
Presolve time: 0.00s
Presolved: 228 rows, 186 columns, 594 nonzeros
Found heuristic solution: objective 1229165.0000
Variable types: 42 continuous, 144 integer (0 binary)
Found heuristic solution: objective 1198245.0000

Root relaxation: objective 1.012257e+06, 59 iterations, 0.00 seconds (0.00 work units)

In [40]:
df = df_results22(dict_N=dict_N,dict_P=dict_P,dict_Nstart=dict_Nstart,dict_XX=dict_Thermique,consommation=consommation)
fig = Results_viz(df,dict_Thermique)
A = 3
fig.update_layout(
    title = f"Répartition de la production électrique dans la journée - {A}"
)
fig.show()
fig.write_html(f"HTML/Répartition de la production électrique dans la journée - {A}.html")


## 4 Planification cyclique

Pour ce problème on modifie la convention $N_{-1}^{(X)}=0$ par $N_{-1}^{(X)}=N_{23}^{(X)}$

In [41]:
model = gp.Model(name="4")

dict_N, dict_Nstart, dict_P = variable_decision_thermique_avec_demarrage(model,dict_Thermique)

model.setObjective(gp.quicksum([dict_N[element[0],element[1]] * (dict_Thermique[element[0]].Cbase - dict_Thermique[element[0]].Pmin*dict_Thermique[element[0]].Cmwh) + dict_P[element[0],element[1]] * dict_Thermique[element[0]].Cmwh + dict_Nstart[element[0],element[1]]*dict_Thermique[element[0]].Cstart for element in product(dict_Thermique,range(24))]), gp.GRB.MINIMIZE)

contraintes_demarrage(model,dict_N,dict_Nstart,dict_Thermique,cyclique=True)
contraintes_puissance_thermique(model,dict_N,dict_P,dict_Thermique)
contraintes_equilibre(model,dict_P,dict_Thermique,consommation)
contraintes_reserve_de_puissance(model,dict_N,dict_Thermique,consommation)


model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 336 rows, 216 columns and 792 nonzeros
Model fingerprint: 0xfaf92924
Variable types: 72 continuous, 144 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 2e+03]
  Bounds range     [5e+00, 1e+01]
  RHS range        [5e+00, 5e+04]
Found heuristic solution: objective 1313700.0000
Presolve removed 105 rows and 27 columns
Presolve time: 0.00s
Presolved: 231 rows, 189 columns, 603 nonzeros
Found heuristic solution: objective 1225165.0000
Variable types: 42 continuous, 147 integer (0 binary)
Found heuristic solution: objective 1192195.0000

Root relaxation: objective 9.855143e+05, 54 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9

In [42]:
df = df_results22(dict_N=dict_N,dict_P=dict_P,dict_Nstart=dict_Nstart,dict_XX=dict_Thermique,consommation=consommation)
fig = Results_viz(df,dict_Thermique)
fig.update_layout(
    title = "Répartition de la production électrique dans la journée - 4"
)
fig.show()
fig.write_html("HTML/Répartition de la production électrique dans la journée - 4.html")


## 5 Centrales hydroélectriques

### 5.1

On ajoute les variables de décisions suivantes:<br>
pour $Y\in[9,14]$ (pour les centrales de 900 MW et 1400MW),<br>
$$
H_t^{(Y)} \in \{0,1\} \text{  , vaut 1 si la centrale $Y$ fonctionne à } t \text{ h 0 sinon}
$$
$$
H_{start,t}^{(Y)} \in \{0,1\} \text{  , vaut 1  si la centrale $Y$ démarre à } t \text{ h 0 sinon}
$$
Avec les contraintes :
$$
H_{start,t}^{(Y)} \leq 1 - H_{t-1}^{(Y)} \text{  ,  S'il y a un démarrage alors la centrale n'était pas allumée}
$$
Autre option :
$$
H_{t}^{(Y)} \leq H_{start,t}^{(Y)}  + H_{t-1}^{(Y)} \text{  ,  Si la centrale fonctionne alors elle était allumée ou elle démarre}
$$
$$
H_{start,t}^{(Y)} \leq H_{t}^{(Y)} \text{  ,  La centrale fonctionne lorsqu'elle est est démarrée}
$$
$$
\sum_X P_t^{(X)} + \sum_Y N_t^{(Y)}P^{(Y)}= D_t \text{  ,  Contrainte équilibre offre-demande}
$$
$$
\sum_X N_t^{(X)}P_{max}^{(X)} + \sum_Y P^{(Y)}\geq D_t\times 1,15 \text{  ,  marges de sécurité}
$$
L'objectif mis à jour devient:
$$
\text{Minimiser} \sum_X \sum_t (P_t^{(X)} - P_{min}^{(X)}N_t^{(X)}) C_{MWh}^{(X)} + N_t^{(X)}C_{base}^{(X)} + N_{start,t}^{(X)}C_{start}^{(X)} + \sum_Y \sum_t H_t^{(Y)}C_{base}^{(Y)} + H_t^{(Y)}C_{start}^{(Y)}
$$

In [43]:
from utils import Centrale_hydro

In [44]:
dict_Hydro = {
    9 : Centrale_hydro(
    name = "Hydro 900 MW",
    P = 900,
    Cstart = 1500,
    Cheure=90,
    debit=0.31
),
    14 : Centrale_hydro(
    name = "Hydro 1400 MW",
    P = 1400,
    Cstart = 1200,
    Cheure=150,
    debit=0.47
)
}

In [45]:
from utils import variable_decision_hydraulique, contraintes_equilibre_avec_hydro, contraintes_reserve_avec_hydro, contraintes_hydraulique

model = gp.Model(name="5")

dict_H, dict_Hstart = variable_decision_hydraulique(model,dict_Hydro)
dict_N, dict_Nstart, dict_P = variable_decision_thermique_avec_demarrage(model,dict_Thermique)

model.setObjective(
    gp.quicksum([dict_N[element[0],element[1]] * (dict_Thermique[element[0]].Cbase - dict_Thermique[element[0]].Pmin*dict_Thermique[element[0]].Cmwh) for element in product(dict_Thermique,range(24))]) + 
    gp.quicksum([dict_P[element[0],element[1]] * dict_Thermique[element[0]].Cmwh for element in product(dict_Thermique,range(24))])+ 
    gp.quicksum([dict_Nstart[element[0],element[1]]*dict_Thermique[element[0]].Cstart for element in product(dict_Thermique,range(24))])+
    gp.quicksum([dict_H[element[0],element[1]] * dict_Hydro[element[0]].Cheure for element in product(dict_Hydro,range(24))])+
    gp.quicksum([dict_Hstart[element[0],element[1]] * dict_Hydro[element[0]].Cstart for element in product(dict_Hydro,range(24))]),
    gp.GRB.MINIMIZE
    )

contraintes_demarrage(model,dict_N,dict_Nstart,dict_Thermique,cyclique=True)
contraintes_puissance_thermique(model,dict_N,dict_P,dict_Thermique)
contraintes_equilibre_avec_hydro(model,dict_H,dict_P,dict_Thermique,dict_Hydro,consommation)
contraintes_reserve_avec_hydro(model,dict_N,dict_Thermique,dict_Hydro,consommation)
contraintes_hydraulique(model,dict_Hstart,dict_H,dict_Hydro)



model.optimize()
        


Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 432 rows, 312 columns and 1080 nonzeros
Model fingerprint: 0x568b7ef7
Variable types: 72 continuous, 240 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 2e+03]
  Bounds range     [1e+00, 1e+01]
  RHS range        [5e+00, 4e+04]
Found heuristic solution: objective 1447255.0000
Presolve removed 120 rows and 0 columns
Presolve time: 0.00s
Presolved: 312 rows, 312 columns, 840 nonzeros
Variable types: 72 continuous, 240 integer (96 binary)
Found heuristic solution: objective 1429430.0000

Root relaxation: objective 8.884163e+05, 90 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 888416.250    0   30 1429430.00 888416.250  37.8

In [46]:
from utils_viz import df_results5, Results_viz5
df = df_results5(dict_N,dict_P,dict_Nstart,dict_Thermique,dict_H,dict_Hstart,dict_Hydro=dict_Hydro,consommation=consommation)
fig = Results_viz5(df,dict_Thermique,dict_Hydro)
A = "5.1"
fig.update_layout(
    title = "Répartition de la production électrique dans la journée - {A}"
)
fig.show()
fig.write_html("HTML/Répartition de la production électrique dans la journée - {A}.html")

### 5.2

On ajoute les variables de décision suivante :
$$
S_t =  \text{Puissance appelée par le pompage à l'instant }t
$$
Puis la contrainte :
$$
\sum_tS_t d^{(S)} = \sum_{t,Y} H_t^{(Y)}d^{(Y)}
$$
Où $d^{(S)}$ représente la hauteur d'eau élevée par MWh et $d^{(Y)}$ la hauteur d'eau prélevée lorsque la centrale $Y$ fonctionne.<br>
On met à jour les contraintes suivante:
$$
\sum_X P_t^{(X)} + \sum_Y H_t^{(Y)}P^{(Y)}-S_t= D_t \text{  ,  Contrainte équilibre offre-demande}
$$

In [47]:
from utils import contraintes_reservoir, contraintes_equilibre_avec_STEP

model = gp.Model(name="5.2")

dict_H, dict_Hstart = variable_decision_hydraulique(model,dict_Hydro)
dict_N, dict_Nstart, dict_P = variable_decision_thermique_avec_demarrage(model,dict_Thermique)
dict_S = {t : model.addVar(lb=0,vtype=gp.GRB.CONTINUOUS,name=f"pompage à l'instant {t}") for t in range(24)}
debit_S = 1/3000 # débit de pompage par MWh ou par MW pendant une heure

model.setObjective(
    gp.quicksum([dict_N[X,t] * (dict_Thermique[X].Cbase - dict_Thermique[X].Pmin*dict_Thermique[X].Cmwh) for X in dict_Thermique for t in range(24)]) + 
    gp.quicksum([dict_P[X,t] * dict_Thermique[X].Cmwh for X in dict_Thermique for t in range(24)])+ 
    gp.quicksum([dict_Nstart[X,t]*dict_Thermique[X].Cstart for X in dict_Thermique for t in range(24)])+
    gp.quicksum([dict_H[Y,t] * dict_Hydro[Y].Cheure for Y in dict_Hydro for t in range(24)])+
    gp.quicksum([dict_Hstart[Y,t] * dict_Hydro[Y].Cstart for Y in dict_Hydro for t in range(24)]),
    gp.GRB.MINIMIZE
    )

contraintes_demarrage(model,dict_N,dict_Nstart,dict_Thermique,cyclique=True)
contraintes_puissance_thermique(model,dict_N,dict_P,dict_Thermique)
contraintes_reservoir(model,dict_S,dict_Hydro,dict_H,debit_S)
contraintes_equilibre_avec_STEP(model,dict_H,dict_P,dict_Thermique,dict_Hydro,dict_S,consommation)
contraintes_reserve_avec_hydro(model,dict_N,dict_Thermique,dict_Hydro,consommation)
contraintes_hydraulique(model,dict_Hstart,dict_H,dict_Hydro)

model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 433 rows, 336 columns and 1176 nonzeros
Model fingerprint: 0x2bb1f118
Variable types: 96 continuous, 240 integer (96 binary)
Coefficient statistics:
  Matrix range     [3e-04, 4e+03]
  Objective range  [1e+00, 2e+03]
  Bounds range     [1e+00, 1e+01]
  RHS range        [5e+00, 4e+04]
Found heuristic solution: objective 1468770.0000
Presolve removed 120 rows and 0 columns
Presolve time: 0.00s
Presolved: 313 rows, 336 columns, 936 nonzeros
Variable types: 96 continuous, 240 integer (96 binary)
Found heuristic solution: objective 1428970.0000

Root relaxation: objective 9.850143e+05, 145 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 985014.286    0   26 1428970.00 985014.286  31.

In [48]:
from utils_viz import df_results52, Results_viz52
df = df_results52(dict_N,dict_P,dict_Nstart,dict_Thermique,dict_H,dict_Hstart,dict_Hydro,dict_S,debit_S,consommation)
fig = Results_viz52(df,dict_Thermique,dict_Hydro)
A = "5.2"
fig.update_layout(
    title = "Répartition de la production électrique dans la journée - {A}"
)
fig.show()
fig.write_html("HTML/Répartition de la production électrique dans la journée - {A}.html")

### 5.3 Paliers de fonctionnement

Les variables de décisions concernant l'hydroélectricité deviennent :
$$
H_t^{(Y,n)} \in \{0,1\} = \text{La centrale hydroélectrique } Y \text{ fonctionne au palier } n \text{ à l'instant } t
$$
$$
H_{start,t}^{(Y)} \in \{0,1\} = \text{  La centrale hydroélectrique $Y$ démarre à l'instant $t$}
$$
Les contraintes suivantes sont modifiées :

$$
\text{Equilibre offre-demande : } \sum_X P_t^{(X)} + \sum_{Y,n} H_t^{(Y,n)}P^{(Y,n)}-S_t= D_t
$$
$$
\text{Réserve de puissance : } \sum_X N_t^{(X)}P_{max}^{(X)} + \sum_Y P^{(Y,n_{max})}\geq D_t\times 1,15
$$
$$
\text{Niveau réservoir : } \sum_t S_t d^{(S)} = \sum_{t,Y,n} H_t^{(Y,n)}d^{(Y,n)}
$$

Les contraintes sur les démarrages et les paliers deviennent :
$$
\text{Un seul palier fonctionne à la fois , }\sum_n H_t^{(Y,n)} \leq 1
$$
$$
\text{ Si un palier fonctionne alors il y avait déjà un palier actif ou la centrale est démarrée , } H_t^{(Y,n)} \leq \sum_n H_{t-1}^{(Y,n)} + H_{start,t}^{(Y)}
$$
Modification de la fonction objectif :
$$
\text{Minimiser} \sum_X \sum_t (P_t^{(X)} - P_{min}^{(X)}N_t^{(X)}) C_{MWh}^{(X)} + N_t^{(X)}C_{base}^{(X)} + N_{start,t}^{(X)}C_{start}^{(X)} + \sum_{Y,n} \sum_t H_t^{(Y,n)}C_{base}^{(Y,n)} + \sum_Y H_{start,t}^{(Y)}C_{start}^{(Y)}
$$
Où $P^{(Y,n)}$ est la puissance de la centrale $Y$ au palier $n$, $d^{(Y,n)}$ est l'abaissement sur une heure quand la centrale $Y$ fonctionne au palier $n$ et $C_{base}^{(Y,n)}$ est le coût du fonctionnement de la centrale $Y$ pendant une heure de fonctionnement au palier $n$

In [49]:
from utils import Centrale_hydro2
from utils import variables_decision_hydraulique_palier
from utils import contraintes_reserve_palier, contraintes_reservoir_palier, contraintes_hydraulique_palier, contraintes_equilibre_palier

model = gp.Model(name="5.3")

dict_Hydro = {
    9 : Centrale_hydro2(
    name = "Hydro 900 MW",
    P = {1 : 900, 2 : 950, 3 : 1000, 4 : 1100},
    Cstart = 1500,
    Cheure= {1 : 90, 2 : 95, 3 : 105, 4 : 120},
    Palier = [1, 2, 3, 4],
    debit= {1 : 0.31, 2 : 0.33, 3 : 0.35, 4 : 0.38}
),
    14 : Centrale_hydro2(
    name = "Hydro 1400 MW",
    P = {1 : 1400, 2 : 1500, 3 : 1600, 4 : 1700},
    Cstart = 1200,
    Cheure= {1 : 150, 2 : 165, 3 : 185, 4 : 210},
    Palier = [1, 2, 3, 4],
    debit= {1 : 0.47, 2 : 0.50, 3 : 0.53, 4 : 0.56}
)
}

dict_H, dict_Hstart = variables_decision_hydraulique_palier(model,dict_Hydro)
dict_N, dict_Nstart, dict_P = variable_decision_thermique_avec_demarrage(model,dict_Thermique)
dict_S = {t : model.addVar(lb=0,vtype=gp.GRB.CONTINUOUS,name=f"pompage à l'instant {t}") for t in range(24)}
debit_S = 1/3000 # débit de pompage par MWh ou par MW pendant une heure

model.setObjective(
    gp.quicksum([dict_N[X,t] * (dict_Thermique[X].Cbase - dict_Thermique[X].Pmin*dict_Thermique[X].Cmwh) for X in dict_Thermique for t in range(24)]) + 
    gp.quicksum([dict_P[X,t] * dict_Thermique[X].Cmwh for X in dict_Thermique for t in range(24)])+ 
    gp.quicksum([dict_Nstart[X,t]*dict_Thermique[X].Cstart for X in dict_Thermique for t in range(24)])+
    gp.quicksum([dict_H[Y,n,t] * dict_Hydro[Y].Cheure[n] for Y in dict_Hydro for t in range(24) for n in dict_Hydro[Y].Palier])+
    gp.quicksum([dict_Hstart[Y,t] * dict_Hydro[Y].Cstart for Y in dict_Hydro for t in range(24)]),
    gp.GRB.MINIMIZE
    )

contraintes_demarrage(model,dict_N,dict_Nstart,dict_Thermique,cyclique=True)
contraintes_puissance_thermique(model,dict_N,dict_P,dict_Thermique)
contraintes_equilibre_palier(model,dict_P,dict_Thermique,dict_H,dict_Hydro,dict_S,consommation)
contraintes_reserve_palier(model,dict_N,dict_Thermique,dict_Hydro,consommation)
contraintes_reservoir_palier(model,dict_H,dict_Hydro,dict_S,debit_S)
contraintes_hydraulique_palier(model,dict_H,dict_Hydro,dict_Hstart)

model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 577 rows, 480 columns and 2184 nonzeros
Model fingerprint: 0x72ec853a
Variable types: 96 continuous, 384 integer (240 binary)
Coefficient statistics:
  Matrix range     [3e-04, 4e+03]
  Objective range  [1e+00, 2e+03]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 4e+04]
Presolve removed 264 rows and 48 columns
Presolve time: 0.00s
Presolved: 313 rows, 432 columns, 1272 nonzeros
Variable types: 96 continuous, 336 integer (192 binary)
Found heuristic solution: objective 1114770.0000
Found heuristic solution: objective 1026475.0000

Root relaxation: objective 9.830546e+05, 174 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 983054.622    0   32 1026475.00 983054.622 

In [50]:
def df_results53(dict_N,dict_P,dict_Nstart,dict_Thermique,dict_H,dict_Hstart,dict_Hydro,dict_S,debit_S,consommation):
    df = pd.DataFrame()
    df["h"] = range(24) 
    df["Consommation (MW)"] = consommation
    df["Production total"] = 0
    df["Coût total"] = 0
    for X in dict_Thermique:
        df[f"Nb centrale {X}"] = [int(dict_N[X,t].X) for t in range(24)]
        df[f"Puissance tot {X}"] = [dict_P[X,t].X for t in range(24)]
        df[f"Coût {X}"] = [dict_N[X,t].X * dict_Thermique[X].Cbase + (dict_P[X,t].X - dict_N[X,t].X * dict_Thermique[X].Pmin) * dict_Thermique[X].Cmwh + dict_Nstart[X,t].X*dict_Thermique[X].Cstart for t in range(24)]
        df["Production total"] += df[f"Puissance tot {X}"]
        df["Coût total"] += df[f"Coût {X}"]
    for Y in dict_Hydro:
        for n in dict_Hydro[Y].Palier:
            df[f"Fonctionnement {Y}, palier {n}"] = [dict_H[Y,n,t].X for t in range(24)]
            df[f"Puissance tot {Y}, palier {n}"] = [dict_H[Y,n,t].X * dict_Hydro[Y].P[n] for t in range(24)]
            df[f"Coût {Y}, palier {n}"] = [dict_H[Y,n,t].X * dict_Hydro[Y].Cheure[n] for t in range(24)]
        df[f"Démarrage {Y}"] = [dict_Hstart[Y,t].X for t in range(24)]
        df[f"Fonctionnement {Y}"] = [sum(dict_H[Y,n,t].X for n in dict_Hydro[Y].Palier) for t in range(24)]
        df[f"Puissance tot {Y}"] = df[[f"Puissance tot {Y}, palier {n}" for n in dict_Hydro[Y].Palier]].sum(1)
        df[f"Coût {Y}"] = df[[f"Coût {Y}, palier {n}" for n in dict_Hydro[Y].Palier]].sum(1) + pd.DataFrame([dict_Hstart[Y,t].X * dict_Hydro[Y].Cstart for t in range(24)])[0]
        df["Production total"] += df[f"Puissance tot {Y}"]
        df["Coût total"] += df[f"Coût {Y}"]
    df["Pompage"] = [-dict_S[t].X for t in range(24)]
    df["Production total"] += df["Pompage"]
    df["Réservoir variation"] = [dict_S[t].X * debit_S - sum([dict_H[Y,n,t].X * dict_Hydro[Y].debit[n] for Y in dict_Hydro for n in dict_Hydro[Y].Palier]) for t in range(24)]
    df["Réservoir"] = df["Réservoir variation"].cumsum()
    df["Coût MWh"] = df["Coût total"]/df["Production total"]
    return df


In [51]:
df = df_results53(dict_N,dict_P,dict_Nstart,dict_Thermique,dict_H,dict_Hstart,dict_Hydro,dict_S,debit_S,consommation)
fig = Results_viz52(df,dict_Thermique,dict_Hydro)
A = "5.3"
fig.update_layout(
    title = "Répartition de la production électrique dans la journée - {A}"
)
fig.show()
fig.write_html("HTML/Répartition de la production électrique dans la journée - {A}.html")

### 5.4 Exclusion pompage et génération hydro

On introduit la variable de décision suivante :
$$
N^{S}_t \in \{0,1\} = \text{La pompage est en fonctionnement}
$$
On ajoute les contraintes suivante :
$$
\text{Si une centrale hydroélectrique est activée alors le pompage est désactivé , } \frac{1}{\text{Nb centrale hydro}}\sum_{Y,n}H_t^{Y,n} \leq 1 - N_t^{(S)}
$$
$$
\text{Si aucune centrale hydroélectrique est allumée alors il peut y avoir du pompage , } S_t \leq M \times  N_t^{(S)}
$$

In [52]:
from utils import contraintes_hydraulique_pompage

model = gp.Model("5.4")

dict_H, dict_Hstart = variables_decision_hydraulique_palier(model,dict_Hydro)
dict_N, dict_Nstart, dict_P = variable_decision_thermique_avec_demarrage(model,dict_Thermique)
dict_S = {t : model.addVar(lb=0,vtype=gp.GRB.CONTINUOUS,name=f"pompage à l'instant {t}") for t in range(24)}
dict_N_s = {t : model.addVar(lb=0,vtype=gp.GRB.BINARY,name=f"fonctionnement pompe à l'instant {t}") for t in range(24)}
debit_S = 1/3000 # débit de pompage par MWh ou par MW pendant une heure
M = sum([dict_Thermique[X].N * dict_Thermique[X].Pmax for X in dict_Thermique]) + sum([dict_Hydro[Y].P[dict_Hydro[Y].palier_max()] for Y in dict_Hydro])

model.setObjective(
    gp.quicksum([dict_N[X,t] * (dict_Thermique[X].Cbase - dict_Thermique[X].Pmin*dict_Thermique[X].Cmwh) for X in dict_Thermique for t in range(24)]) + 
    gp.quicksum([dict_P[X,t] * dict_Thermique[X].Cmwh for X in dict_Thermique for t in range(24)])+ 
    gp.quicksum([dict_Nstart[X,t]*dict_Thermique[X].Cstart for X in dict_Thermique for t in range(24)])+
    gp.quicksum([dict_H[Y,n,t] * dict_Hydro[Y].Cheure[n] for Y in dict_Hydro for t in range(24) for n in dict_Hydro[Y].Palier])+
    gp.quicksum([dict_Hstart[Y,t] * dict_Hydro[Y].Cstart for Y in dict_Hydro for t in range(24)]),
    gp.GRB.MINIMIZE
    )

contraintes_demarrage(model,dict_N,dict_Nstart,dict_Thermique,cyclique=True)
contraintes_puissance_thermique(model,dict_N,dict_P,dict_Thermique)
contraintes_equilibre_palier(model,dict_P,dict_Thermique,dict_H,dict_Hydro,dict_S,consommation)
contraintes_reserve_palier(model,dict_N,dict_Thermique,dict_Hydro,consommation)
contraintes_reservoir_palier(model,dict_H,dict_Hydro,dict_S,debit_S)
contraintes_hydraulique_palier(model,dict_H,dict_Hydro,dict_Hstart)
contraintes_hydraulique_pompage(model,dict_H,dict_Hydro,dict_N_s,dict_S,M)

model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 625 rows, 504 columns and 2448 nonzeros
Model fingerprint: 0x2871af22
Variable types: 96 continuous, 408 integer (264 binary)
Coefficient statistics:
  Matrix range     [3e-04, 6e+04]
  Objective range  [1e+00, 2e+03]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 4e+04]
Presolve removed 288 rows and 48 columns
Presolve time: 0.00s
Presolved: 337 rows, 456 columns, 1368 nonzeros
Variable types: 96 continuous, 360 integer (216 binary)
Found heuristic solution: objective 1155935.0000
Found heuristic solution: objective 1037700.0000
Found heuristic solution: objective 1026475.0000

Root relaxation: objective 9.830546e+05, 202 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0 

In [53]:
df = df_results53(dict_N,dict_P,dict_Nstart,dict_Thermique,dict_H,dict_Hstart,dict_Hydro,dict_S,debit_S,consommation)
fig = Results_viz52(df,dict_Thermique,dict_Hydro)
A = "5.4"
fig.update_layout(
    title = "Répartition de la production électrique dans la journée - {A}"
)
fig.show()
fig.write_html("HTML/Répartition de la production électrique dans la journée - {A}.html")

## 6 Désagrégation

## 6.1 Individualisation des centrales

Variables de décision :
Pour une type de centrale $X \in [A,B,C]$, $k$ l'identification de la $k$-ieme centrale, et une heure de la journée $t\in {1,\dots,24}$, on définit :
$$
N_t^{(X,k)} \in\{0,1\} \text{La $k$-ieme centrale $X$ fonctionne à $t$}
$$
$$
N_{start,t}^{(X,k)} \in\{0,1\} \text{La $k$-ieme centrale $X$ est démarrée à $t$}
$$
$$
P_t^{(X,k)} = \text{Puissance de la $k$-ième centrale $X$ à l'instant $t$}
$$
Contraintes :
$$
N_t^{(X,k)} P_{min}^{(X)} \leq P_t^{(X,k)} \leq N_t^{(X,k)} P_{max}^{(X)} \text{ ,  Contraintes sur la puissance totale de chaque centrale}
$$
$$
\text{Equilibre offre-demande : } \sum_X \sum_{k=1}^{N^{(X)}} P_t^{(X,k)} + \sum_{Y,n} H_t^{(Y,n)}P^{(Y,n)}-S_t= D_t
$$
$$
\text{Réserve de puissance : } \sum_X \sum_{k=1}^{N^{(X)}} N_t^{(X,k)}P_{max}^{(X)} + \sum_Y P^{(Y,n_{max})}\geq D_t\times 1,15
$$
Objectif :
$$
\text{Minimiser} \sum_X \sum_{k=1}^{N^{(X)}} \sum_t (P_t^{(X,k)} - P_{min}^{(X)}N_t^{(X,k)}) C_{MWh}^{(X)} + N_t^{(X,k)}C_{base}^{(X)} + N_{start,t}^{(X,k)}C_{start}^{(X)} + \sum_{Y,n} \sum_t H_t^{(Y,n)}C_{base}^{(Y,n)} + \sum_Y H_{start,t}^{(Y)}C_{start}^{(Y)}
$$

In [25]:
from utils import variables_decision_desagregation_thermique
from utils import contraintes_thermique_desagregation, contraintes_equilibre_desagregation, contraintes_reserve_desagregation

model = gp.Model("6.1")

dict_N, dict_P, dict_Nstart = variables_decision_desagregation_thermique(model,dict_Thermique)
dict_H, dict_Hstart = variables_decision_hydraulique_palier(model,dict_Hydro)
dict_S = {t : model.addVar(lb=0,vtype=gp.GRB.CONTINUOUS,name=f"pompage à l'instant {t}") for t in range(24)}
dict_N_s = {t : model.addVar(lb=0,vtype=gp.GRB.BINARY,name=f"fonctionnement pompe à l'instant {t}") for t in range(24)}
debit_S = 1/3000 # débit de pompage par MWh ou par MW pendant une heure
M = sum([dict_Thermique[X].N * dict_Thermique[X].Pmax for X in dict_Thermique]) + sum([dict_Hydro[Y].P[dict_Hydro[Y].palier_max()] for Y in dict_Hydro])

contraintes_reservoir_palier(model,dict_H,dict_Hydro,dict_S,debit_S)
contraintes_hydraulique_palier(model,dict_H,dict_Hydro,dict_Hstart)
contraintes_hydraulique_pompage(model,dict_H,dict_Hydro,dict_N_s,dict_S,M)
contraintes_thermique_desagregation(model,dict_P,dict_N,dict_Nstart,dict_Thermique)
contraintes_equilibre_desagregation(model,dict_P,dict_Thermique,dict_H,dict_Hydro,dict_S,consommation)
contraintes_reserve_desagregation(model,dict_N,dict_Thermique,dict_Hydro,consommation)

model.setObjective(
    gp.quicksum([dict_N[X,k,t] * (dict_Thermique[X].Cbase - dict_Thermique[X].Pmin*dict_Thermique[X].Cmwh) for X in dict_Thermique for k in range(dict_Thermique[X].N) for t in range(24)]) + 
    gp.quicksum([dict_P[X,k,t] * dict_Thermique[X].Cmwh for X in dict_Thermique for k in range(dict_Thermique[X].N) for t in range(24)])+ 
    gp.quicksum([dict_Nstart[X,k,t]*dict_Thermique[X].Cstart for X in dict_Thermique for k in range(dict_Thermique[X].N) for t in range(24)])+
    gp.quicksum([dict_H[Y,n,t] * dict_Hydro[Y].Cheure[n] for Y in dict_Hydro for t in range(24) for n in dict_Hydro[Y].Palier])+
    gp.quicksum([dict_Hstart[Y,t] * dict_Hydro[Y].Cstart for Y in dict_Hydro for t in range(24)]),
    gp.GRB.MINIMIZE
    )

model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2281 rows, 2232 columns and 7488 nonzeros
Model fingerprint: 0x18039317
Variable types: 672 continuous, 1560 integer (1560 binary)
Coefficient statistics:
  Matrix range     [3e-04, 6e+04]
  Objective range  [1e+00, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+04]
Presolve removed 216 rows and 48 columns
Presolve time: 0.01s
Presolved: 2065 rows, 2184 columns, 6552 nonzeros
Variable types: 672 continuous, 1512 integer (1512 binary)
Found heuristic solution: objective 1047900.0000
Found heuristic solution: objective 1031950.0000

Root relaxation: objective 9.830120e+05, 1384 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 983011.964    0   31 1031950.00 

### 6.3 Précision du profil de la demande

In [26]:
consommation = np.array([17,15,14,13,13,18,29,32,29,27,25,25,24,24,25,38,42,40,33,31,27,26,24,21])*1000

model = gp.Model("6.2")

dict_N, dict_P, dict_Nstart = variables_decision_desagregation_thermique(model,dict_Thermique)
dict_H, dict_Hstart = variables_decision_hydraulique_palier(model,dict_Hydro)
dict_S = {t : model.addVar(lb=0,vtype=gp.GRB.CONTINUOUS,name=f"pompage à l'instant {t}") for t in range(24)}
dict_N_s = {t : model.addVar(lb=0,vtype=gp.GRB.BINARY,name=f"fonctionnement pompe à l'instant {t}") for t in range(24)}
debit_S = 1/3000 # débit de pompage par MWh ou par MW pendant une heure
M = sum([dict_Thermique[X].N * dict_Thermique[X].Pmax for X in dict_Thermique]) + sum([dict_Hydro[Y].P[dict_Hydro[Y].palier_max()] for Y in dict_Hydro])

contraintes_reservoir_palier(model,dict_H,dict_Hydro,dict_S,debit_S)
contraintes_hydraulique_palier(model,dict_H,dict_Hydro,dict_Hstart)
contraintes_hydraulique_pompage(model,dict_H,dict_Hydro,dict_N_s,dict_S,M)
contraintes_thermique_desagregation(model,dict_P,dict_N,dict_Nstart,dict_Thermique)
contraintes_equilibre_desagregation(model,dict_P,dict_Thermique,dict_H,dict_Hydro,dict_S,consommation)
contraintes_reserve_desagregation(model,dict_N,dict_Thermique,dict_Hydro,consommation)

model.setObjective(
    gp.quicksum([dict_N[X,k,t] * (dict_Thermique[X].Cbase - dict_Thermique[X].Pmin*dict_Thermique[X].Cmwh) for X in dict_Thermique for k in range(dict_Thermique[X].N) for t in range(24)]) + 
    gp.quicksum([dict_P[X,k,t] * dict_Thermique[X].Cmwh for X in dict_Thermique for k in range(dict_Thermique[X].N) for t in range(24)])+ 
    gp.quicksum([dict_Nstart[X,k,t]*dict_Thermique[X].Cstart for X in dict_Thermique for k in range(dict_Thermique[X].N) for t in range(24)])+
    gp.quicksum([dict_H[Y,n,t] * dict_Hydro[Y].Cheure[n] for Y in dict_Hydro for t in range(24) for n in dict_Hydro[Y].Palier])+
    gp.quicksum([dict_Hstart[Y,t] * dict_Hydro[Y].Cstart for Y in dict_Hydro for t in range(24)]),
    gp.GRB.MINIMIZE
    )

model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2281 rows, 2232 columns and 7488 nonzeros
Model fingerprint: 0x763ca6d9
Variable types: 672 continuous, 1560 integer (1560 binary)
Coefficient statistics:
  Matrix range     [3e-04, 6e+04]
  Objective range  [1e+00, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+04]
Presolve removed 216 rows and 48 columns
Presolve time: 0.01s
Presolved: 2065 rows, 2184 columns, 6552 nonzeros
Variable types: 672 continuous, 1512 integer (1512 binary)
Found heuristic solution: objective 1047500.0000

Root relaxation: objective 9.841490e+05, 1279 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 984149.048    0   22 1047500.00 984149.048  6.05%     -    0s
H    0     0       

### 6.4 Discrétisation à un pas de deux heures

In [27]:
consommation = np.array([16,16,14,14,16,16,30,30,28,28,25,25,24,24,33,33,41,41,32,32,26,26,22,22])*1000

model = gp.Model("6.2")

dict_N, dict_P, dict_Nstart = variables_decision_desagregation_thermique(model,dict_Thermique)
dict_H, dict_Hstart = variables_decision_hydraulique_palier(model,dict_Hydro)
dict_S = {t : model.addVar(lb=0,vtype=gp.GRB.CONTINUOUS,name=f"pompage à l'instant {t}") for t in range(24)}
dict_N_s = {t : model.addVar(lb=0,vtype=gp.GRB.BINARY,name=f"fonctionnement pompe à l'instant {t}") for t in range(24)}
debit_S = 1/3000 # débit de pompage par MWh ou par MW pendant une heure
M = sum([dict_Thermique[X].N * dict_Thermique[X].Pmax for X in dict_Thermique]) + sum([dict_Hydro[Y].P[dict_Hydro[Y].palier_max()] for Y in dict_Hydro])

contraintes_reservoir_palier(model,dict_H,dict_Hydro,dict_S,debit_S)
contraintes_hydraulique_palier(model,dict_H,dict_Hydro,dict_Hstart)
contraintes_hydraulique_pompage(model,dict_H,dict_Hydro,dict_N_s,dict_S,M)
contraintes_thermique_desagregation(model,dict_P,dict_N,dict_Nstart,dict_Thermique)
contraintes_equilibre_desagregation(model,dict_P,dict_Thermique,dict_H,dict_Hydro,dict_S,consommation)
contraintes_reserve_desagregation(model,dict_N,dict_Thermique,dict_Hydro,consommation)

model.setObjective(
    gp.quicksum([dict_N[X,k,t] * (dict_Thermique[X].Cbase - dict_Thermique[X].Pmin*dict_Thermique[X].Cmwh) for X in dict_Thermique for k in range(dict_Thermique[X].N) for t in range(24)]) + 
    gp.quicksum([dict_P[X,k,t] * dict_Thermique[X].Cmwh for X in dict_Thermique for k in range(dict_Thermique[X].N) for t in range(24)])+ 
    gp.quicksum([dict_Nstart[X,k,t]*dict_Thermique[X].Cstart for X in dict_Thermique for k in range(dict_Thermique[X].N) for t in range(24)])+
    gp.quicksum([dict_H[Y,n,t] * dict_Hydro[Y].Cheure[n] for Y in dict_Hydro for t in range(24) for n in dict_Hydro[Y].Palier])+
    gp.quicksum([dict_Hstart[Y,t] * dict_Hydro[Y].Cstart for Y in dict_Hydro for t in range(24)]),
    gp.GRB.MINIMIZE
    )

model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2281 rows, 2232 columns and 7488 nonzeros
Model fingerprint: 0x35a75956
Variable types: 672 continuous, 1560 integer (1560 binary)
Coefficient statistics:
  Matrix range     [3e-04, 6e+04]
  Objective range  [1e+00, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+04]
Presolve removed 216 rows and 48 columns
Presolve time: 0.01s
Presolved: 2065 rows, 2184 columns, 6552 nonzeros
Variable types: 672 continuous, 1512 integer (1512 binary)
Found heuristic solution: objective 1044200.0000

Root relaxation: objective 9.869988e+05, 1266 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 986998.810    0   24 1044200.00 986998.810  5.48%     -    0s
H    0     0       

## 7 Disponibilité variable

### 7.1 Maintenance nocturne des centrales A

On ajoute la contrainte :
$$
\forall t \in \{0,\dots,6\}\cup\{18,\dots,23\},\sum_{k=1}^{N^{(A)}}N_t^{(A,k)} \leq 10
$$

## 8 Contraintes de rampe

### 8.1 Rampe croissante

$$
\forall t, \forall X , \forall k \in \{0,\dots,N^{(X)}\}, P_{t}^{(X,k)}-P_{t-1}^{(X,k)}\leq R_{montée}^{(X)}
$$

### 8.2 Rampe démarrage

$$
\forall t, \forall X , \forall k \in \{0,\dots,N^{(X)}\}, P_{t}^{(X,k)}-P_{t-1}^{(X,k)}\leq R_{montée}^{(X)} + N_{start,t}{(X,k)}(R_{démarrage}^{(X)}-R_{montée}^{(X)})
$$

### 8.3 Rampe décroissante et d'arrêt

$$
\forall t, \forall X , \forall k \in \{0,\dots,N^{(X)}\}, P_{t}^{(X,k)}-P_{t-1}^{(X,k)}\geq R^{(X)}_{descente}
$$

## 9 Prévention de l'usure

### 9.1 Durée minimale d'activitée

$$
\forall t,\forall t' \in \{1,\dots,8\}, \forall k, H_{start,t}^{(X,k)}\leq H_{t+t'}^{(X,k)}
$$

### 9.2 Durée minimale d'arrêt 

$$
\forall t,\forall t' \in \{1,\dots,8\}, \forall k, \frac{H_{t-1}^{(X,k)}-H_{t}^{(X,k)}+1}{2}\geq H_{t+t'}^{(X,k)}
$$